## import pickle

In [4]:
import pickle
import pandas as pd

# Load the model and label encoder from the pickle file
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)


loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

## .predict

In [5]:
new_data = [[3.0000, 3.0000, 1.638227, -15, 'slownfastp']]

new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
new_df['process'] = loaded_encoder.transform(new_df['process'])

prediction = loaded_model.predict(new_df)

print(f'Predicted vinn: {prediction[0]}')
print(new_df)

Predicted vinn: 0.5328154581304556
   vdd   pd      vinp  temperature  process
0  3.0  3.0  1.638227          -15        2


## manual prediction

In [6]:
def manual_predict(model, features):
    predictions = []
    for tree in model.estimators_:
        tree_prediction = traverse_tree(tree.tree_, features)
        predictions.append(tree_prediction)
    return sum(predictions) / len(predictions)

def traverse_tree(tree, features):
    node_id = 0
    while True:
        feature = tree.feature[node_id]
        if feature == -2:  # Leaf node
            return tree.value[node_id][0][0]
        else:
            if features[feature] <= tree.threshold[node_id]:
                next_node_id = tree.children_left[node_id]
            else:
                next_node_id = tree.children_right[node_id]
            if next_node_id == -1:
                return tree.value[node_id][0][0]
            else:
                node_id = next_node_id


In [7]:
# Input data for prediction
new_data = [[3.0000, 3.0000, 1.638227, -15, 'slownslowp']]
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
new_df['process'] = loaded_encoder.transform(new_df['process'])

print(new_df)
# Manual prediction
manual_prediction = manual_predict(loaded_model, new_df.values[0])
print(f"Manual Prediction for input features: {manual_prediction}")


   vdd   pd      vinp  temperature  process
0  3.0  3.0  1.638227          -15        3
Manual Prediction for input features: 0.7550352739242591


In [ ]:
import pickle
import pandas as pd

with open('Random_Forest.pkl', 'rb') as file:
    model_data = pickle.load(file)



loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test\fastnfastp_3.6V_45.csv")
time = data['time']
vinn = data['vinn']

plt.plot(time, vinn, label='Actual', color='b')

new_data = data[['vdd', 'pd', 'vinp', 'temperature','process']]
# new_data = loaded_encoder.transform(new_data['process'])
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature','process'])
new_df['process'] = loaded_encoder.transform(new_df['process'])

prediction = loaded_model.predict(new_df)

print(f'Predicted vinn: {prediction[0]}')
print(new_df)
plt.plot(time, prediction, label='Predicted', color='red')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Output')
plt.title('Waveform Plot Actual vs Predicted')
plt.tight_layout()
plt.show()


In [ ]:
import pickle
import pandas as pd

with open('Random_Forest_20.pkl', 'rb') as file:
    model_data = pickle.load(file)



loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test\fastnfastp_3.6V_45.csv")
time = data['time']
vinn = data['vinn']

plt.plot(time, vinn, label='Actual', color='b')

new_data = data[['vdd', 'pd', 'vinp', 'temperature','process']]
# new_data = loaded_encoder.transform(new_data['process'])
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature','process'])
new_df['process'] = loaded_encoder.transform(new_df['process'])

prediction = loaded_model.predict(new_df)

print(f'Predicted vinn: {prediction[0]}')
print(new_df)
plt.plot(time, prediction, label='Predicted', color='red')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Output')
plt.title('Waveform Plot Actual vs Predicted')
plt.tight_layout()
plt.show()


## if-else Rule

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import _tree
import numpy as np
import pickle

def get_rules(tree, feature_names, class_names):
    try:
        tree_ = tree.tree_
        feature_name = [
            feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
            for i in tree_.feature
        ]

        paths = []
        path = []

        def recurse(node, path, paths):
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                p1, p2 = list(path), list(path)
                p1 += [f"({name} <= {np.round(threshold, 3)})"]
                recurse(tree_.children_left[node], p1, paths)
                p2 += [f"({name} > {np.round(threshold, 3)})"]
                recurse(tree_.children_right[node], p2, paths)
            else:
                path += [(tree_.value[node], tree_.n_node_samples[node])]
                paths += [path]

        recurse(0, path, paths)

        samples_count = [p[-1][1] for p in paths]
        ii = list(np.argsort(samples_count))
        paths = [paths[i] for i in reversed(ii)]

        rules = []
        for path in paths:
            rule = "if "

            for p in path[:-1]:
                if rule != "if ":
                    rule += " and "
                rule += str(p)
            
            if class_names is None:
                rule += ": return " + str(np.round(path[-1][0][0][0], 3))
            else:
                classes = path[-1][0][0]
                l = np.argmax(classes)
                rule += f"class: {class_names[l]} (proba: {np.round(100.0 * classes[l] / np.sum(classes), 2)}%)"

            rules += [rule]

        return rules
    except (AttributeError, NameError) as e:
        print(f"Error extracting rules: {e}")
        return None


def get_all_rules(forest, feature_names, class_names):
    all_rules = []
    for tree in forest.estimators_:
        rules = get_rules(tree, feature_names, class_names)
        if rules:
            all_rules.append(rules)
    return all_rules



# Load the model
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

print("Random Forest Regressor model loaded successfully.")
    
  
class_names = None  

# Extract rules from the Random Forest
all_rules = get_all_rules(model, feature_names, class_names)

# Print the extracted rules
for i, tree_rules in enumerate(all_rules):
    print(f"\nRules from Tree {i+1}")
    for rule in tree_rules:
        print(rule)

# Write rules to a file if needed
with open('output.txt', 'w') as file:
    for i, tree_rules in enumerate(all_rules):
        file.write(f"\nRules from Tree {i+1}\n")
        for rule in tree_rules:
            file.write(rule + '\n')

print("Rules have been extracted and written to output.txt.")
    



## Read Parameter

In [3]:
import re
import pickle

def extract_words_after_parentheses(pseudocode):
    
    matches = re.findall(r'\(\s*([a-zA-Z]+)\s*>', pseudocode)
    
    extracted_set = set(matches)
    
    return extracted_set

def read_pseudocode_from_file(file_path):
    with open(file_path, 'r') as file:
        pseudocode = file.read()
    return pseudocode

file_path = "output.txt"
pseudocode = read_pseudocode_from_file(file_path)

result_set = extract_words_after_parentheses(pseudocode)

print(result_set)



def export_set(my_set, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(my_set, file)

export_file_path = 'exported_set.pkl'

export_set(result_set, export_file_path)


{'process', 'vdd', 'temperature', 'pd', 'vinp'}


## Generate Function

In [5]:

import pickle

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

import_file_path = 'exported_set.pkl'


def generate_python_code_from_pseudocode(pseudocode_lines, input_parameters):
    python_code = ""
    rule_count = 1
    capture_lines = False

    for line in pseudocode_lines:
        if f"Rules from Tree {rule_count}" in line:
            parameter_string = ', '.join(f"{param}=None" for param in input_parameters)
            python_code += f"\ndef decision_tree_rule_{rule_count}({parameter_string}):" + "\n"
            rule_count += 1
            capture_lines = True

        elif capture_lines and line.strip():  
            indented_code_block = "    " + line.replace('return', '\n        return')
            python_code += f"{indented_code_block}" + "\n"

    return python_code


input_parameters = import_set(import_file_path)
print("Imported Set:", input_parameters)

file_path = "output.txt" 
with open(file_path, 'r') as file:
    pseudocode_lines = file.readlines()


python_code = generate_python_code_from_pseudocode(pseudocode_lines, input_parameters)
print("Python Code generated")

output_file_path = "fun.py"  
with open(output_file_path, 'w') as code_file:
    code_file.write(python_code)

print(f"Python code has been saved to: {output_file_path}")


Imported Set: {'process', 'vdd', 'temperature', 'pd', 'vinp'}
Python Code generated
Python code has been saved to: fun.py


## predict function

In [ ]:

import numpy as np
import pickle

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

import_file_path = 'exported_set.pkl'




def evaluate_all_conditions(**kwargs):
    results = []

    function_file_path = "fun.py"

    with open(function_file_path, 'r') as file:
        functions_code = file.read()

    for i in range(1,101):
        function_name = f"decision_tree_rule_{i}"
        exec(functions_code)
        function = locals()[function_name]
        result = function(**kwargs) or 0
        results.append(result)

    return results

with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']


input_parameters = import_set(import_file_path)

print("Imported Set:", input_parameters)

## input  values
new_data = [[3.0000, 3.0000, 1.638227, 85, 'slownslowp']]
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
new_df['process'] = loaded_encoder.transform(new_df['process'])

vinp_value = new_df['vinp'][0]
pd_value = new_df['pd'][0]
vdd_value = new_df['vdd'][0]
process_value = new_df['process'][0]
temperature_value =new_df['temperature'][0]


input_values = {'vinp': vinp_value, 'pd': pd_value, 'vdd': vdd_value, 'process':process_value,'temperature':temperature_value}

result_list = evaluate_all_conditions(**input_values)

print("Length:", len(result_list))
print("Results:", result_list)
print("Average:", np.average(result_list))


In [ ]:
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_text
import pandas as pd
import numpy as np
from sklearn.tree import _tree

def load_model(pkl_path):
    try:
        with open(pkl_path, 'rb') as file:
            model_data = pickle.load(file)
        return model_data
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return None

pkl_path = 'Random_forest_model.pkl'
loaded_model_data = load_model(pkl_path)

if loaded_model_data is not None and isinstance(loaded_model_data['model'], RandomForestRegressor):
    print("RandomForestRegressor model loaded successfully.")

    loaded_model = loaded_model_data['model']
    feature_names = loaded_model_data['feature_names']
    
    print("Original Feature Names:", feature_names)

    num_features = loaded_model.estimators_[0].tree_.n_features
    print("Number of Features:", num_features)
    
    # Define feature names if not available
    feature_names = [f'Feature_{i}' for i in range(1, num_features + 1)]

    for i, tree in enumerate(loaded_model.estimators_):
        tree_text = export_text(tree, feature_names=feature_names)
        print(f"Decision Tree {i + 1}:\n{tree_text}\n{'='*50}\n")

else:
    print("Failed to load the RandomForestRegressor model.")


In [ ]:
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Load the model and label encoder from the pickle file
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

# Prepare new data
new_data = [[3.0000, 3.0000, 1.638227, 85, 'slownfastp']]
new_df = pd.DataFrame(new_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])

# Transform categorical features using the loaded label encoder
new_df['process'] = loaded_encoder.transform(new_df['process'])

# Make the prediction
prediction = loaded_model.predict(new_df[feature_names])

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, prediction)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_true, prediction))

# Calculate Signal-to-Noise Ratio (SNR)
signal_power = np.mean(y_true**2)
noise_power = np.mean((y_true - prediction)**2)
snr = 10 * np.log10(signal_power / noise_power)

# Display the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Signal-to-Noise Ratio (SNR): {snr}')


In [ ]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle
import numpy as np


with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']


folder_path = r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test"
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(folder_path, filename)
        csv_data = pd.read_csv(csv_path)
        for index, row in csv_data.iterrows():
            
            input_data = np.array(row[['vdd','pd','vinp','temperature','process']]).reshape(1, -1) 
            new_df = pd.DataFrame(input_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
            new_df['process'] = loaded_encoder.transform(new_df['process'])
            
            input_data = new_df
            prediction = loaded_model.predict(input_data)
            print(f'Prediction for row {index} in file {filename}: {prediction[0]}')


predictions = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(folder_path, filename)
        csv_data = pd.read_csv(csv_path)
        for index, row in csv_data.iterrows():
            input_data = np.array(row[['vdd','pd','vinp','temperature','process']]).reshape(1, -1)  
            new_df = pd.DataFrame(input_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
            new_df['process'] = loaded_encoder.transform(new_df['process'])
            
            input_data = new_df
            prediction = loaded_model.predict(input_data)
            predictions.append(prediction[0])

y_test = csv_data['vinn'] 
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')


In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle
import numpy as np
import matplotlib.pyplot as plt

with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

folder_path = r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test"

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(folder_path, filename)
        csv_data = pd.read_csv(csv_path)
        
        # Extract specific columns for input and output waveforms
        input_cols = ['vdd', 'pd', 'vinp', 'temperature','process']
        output_col = 'vinn'
        
        for index, row in csv_data.iterrows():
            # Extract input data
            input_data = np.array(row[['vdd','pd','vinp','temperature','process']]).reshape(1, -1) 
            new_df = pd.DataFrame(input_data, columns=['vdd', 'pd', 'vinp', 'temperature', 'process'])
            new_df['process'] = loaded_encoder.transform(new_df['process'])
            input_data = new_df
            # Predict
            prediction = loaded_model.predict(input_data)
            
            # Print prediction
            print(f'Prediction for row {index} in file {filename}: {prediction[0]}')
            
            # Plot input waveform
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(input_cols)), input_data[0], marker='o', label='Input Waveform')
            plt.xticks(range(len(input_data)), input_cols)
            plt.xlabel('Features')
            plt.ylabel('Values')
            plt.title('Input Waveform')
            plt.legend()
            plt.show()

            # Plot output waveform (actual vs. predicted)
            plt.figure(figsize=(10, 6))
            plt.plot(row.index, row[output_col], label='Actual')
            plt.plot(row.index, [prediction[0]] * len(row), label='Predicted')
            plt.xlabel('Time')
            plt.ylabel('Voltage')
            plt.title('Output Waveform (Actual vs. Predicted)')
            plt.legend()
            plt.show()

# Calculate RMSE
y_test = csv_data[output_col] 
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle
import numpy as np

# Load the model and label encoder from the pickle file
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

# Define the folder path containing CSV files
folder_path = r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test"

# Lists to store predictions and actual values
predictions = []
actual_values = []

# Iterate over each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(folder_path, filename)
        csv_data = pd.read_csv(csv_path)
        
        # Iterate over each row in the CSV file
        for index, row in csv_data.iterrows():
            # Extract input features
            input_data = np.array(row[['vdd', 'pd', 'vinp', 'temperature', 'process']]).reshape(1, -1)
            # Transform categorical feature 'process' using loaded label encoder
            input_data[0, -1] = loaded_encoder.transform([input_data[0, -1]])
            
            # Make prediction
            prediction = loaded_model.predict(input_data)
            predictions.append(prediction[0])
            
            # Print prediction for the row
            print(f'Prediction for row {index} in file {filename}: {prediction[0]}')

            # Plot input waveform
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(input_data.flatten())), input_data.flatten(), marker='o', label='Input Waveform')
            plt.xlabel('Features')
            plt.ylabel('Values')
            plt.title('Input Waveform')
            plt.legend()
            plt.show()

            # Plot output waveform (actual vs. predicted)
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(row['vinn'])), row['vinn'], label='Actual')
            plt.plot(range(len(row['vinn'])), [prediction[0]] * len(row['vinn']), label='Predicted')
            plt.xlabel('Time')
            plt.ylabel('Voltage')
            plt.title('Output Waveform (Actual vs. Predicted)')
            plt.legend()
            plt.show()

            # Append actual and predicted values
            actual_values.append(row['vinn'])
            predictions.append(prediction[0])

# Calculate RMSE
rmse = sqrt(mean_squared_error(actual_values, predictions))
print(f'Root Mean Squared Error: {rmse}')


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle
import numpy as np

# Load the model and label encoder from the pickle file
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

# Define the folder path containing CSV files
folder_path = r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test"

# Lists to store predictions and actual values
predictions = []
actual_values = []

# Iterate over each CSV file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        csv_path = os.path.join(folder_path, filename)
        csv_data = pd.read_csv(csv_path)
        
        # Iterate over each row in the CSV file
        for index, row in csv_data.iterrows():
            # Extract input features
            input_data = np.array(row[['vdd', 'pd', 'vinp', 'temperature', 'process']]).reshape(1, -1)
            # Transform categorical feature 'process' using loaded label encoder
            input_data[0, -1] = loaded_encoder.transform([input_data[0, -1]])
            
            # Make prediction
            prediction = loaded_model.predict(input_data)
            predictions.append(prediction[0])
            
            # Print prediction for the row
            print(f'Prediction for row {index} in file {filename}: {prediction[0]}')

            # Plot input waveform
            plt.figure(figsize=(10, 6))
            plt.plot(range(len(input_data.flatten())), input_data.flatten(), marker='o', label='Input Waveform')
            plt.xlabel('Features')
            plt.ylabel('Values')
            plt.title('Input Waveform')
            plt.legend()
            plt.show()

            # Plot output waveform (actual vs. predicted)
            plt.figure(figsize=(10, 6))
            actual_value = row['vinn']
            if isinstance(actual_value, (list, np.ndarray)):
                actual_values = actual_value
            else:
                actual_values = [actual_value]
            plt.plot(range(len(actual_values)), actual_values, label='Actual')
            plt.plot(range(len(actual_values)), [prediction[0]] * len(actual_values), label='Predicted')
            plt.xlabel('Time')
            plt.ylabel('Voltage')
            plt.title('Output Waveform (Actual vs. Predicted)')
            plt.legend()
            plt.show()

            # Append actual and predicted values
            predictions.append(prediction[0])

# Calculate RMSE
rmse = sqrt(mean_squared_error(actual_values, predictions))
print(f'Root Mean Squared Error: {rmse}')


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import pickle
import numpy as np
from matplotlib.animation import FuncAnimation

# Load the model and label encoder from the pickle file
with open('Random_forest_model.pkl', 'rb') as file:
    model_data = pickle.load(file)

loaded_model = model_data['model']
loaded_encoder = model_data['label_encoder']
feature_names = model_data['feature_names']

# Define the folder path containing CSV files
folder_path = r"C:\Users\ASUS\OneDrive\Desktop\FourthSEM\test"

# Create a figure and axis for the waveform plot
fig, ax = plt.subplots(figsize=(10, 6))

# Initialize empty lists to store predictions and actual values
predictions = []
actual_values = []

# Define a function to update the plot
def update_plot(frame):
    ax.clear()  # Clear the axis for new plot
    
    # Read the CSV file for the current frame
    filename = os.listdir(folder_path)[frame]
    csv_path = os.path.join(folder_path, filename)
    csv_data = pd.read_csv(csv_path)
    
    # Iterate over each row in the CSV file
    for index, row in csv_data.iterrows():
        # Extract input features
        input_data = np.array(row[['vdd', 'pd', 'vinp', 'temperature', 'process']]).reshape(1, -1)
        # Transform categorical feature 'process' using loaded label encoder
        input_data[0, -1] = loaded_encoder.transform([input_data[0, -1]])
        
        # Make prediction
        prediction = loaded_model.predict(input_data)
        predictions.append(prediction[0])
        
        # Print prediction for the row
        print(f'Prediction for row {index} in file {filename}: {prediction[0]}')

        # Plot input waveform
        ax.plot(range(len(input_data.flatten())), input_data.flatten(), marker='o', label='Input Waveform')
        
        # Append actual value
        actual_values.append(row['vinn'])
        
        # Plot output waveform (actual vs. predicted)
        ax.plot(range(len(actual_values)), actual_values, label='Actual')
        ax.plot(range(len(predictions)), predictions, label='Predicted')
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Voltage')
        ax.set_title('Output Waveform (Actual vs. Predicted)')
        ax.legend()

# Create an animation
animation = FuncAnimation(fig, update_plot, frames=len(os.listdir(folder_path)), repeat=False)

# Show the animation
plt.show()

# Calculate RMSE
rmse = sqrt(mean_squared_error(actual_values, predictions))
print(f'Root Mean Squared Error: {rmse}')
